In [19]:
import numpy as np
import pandas as pd
#import tensorflow as tf


import contractions
import os
from collections import Counter
from bs4 import BeautifulSoup
import torch
from torch.utils.data import DataLoader, Dataset

# from transformers import BertForSequenceClassification, BertTokenizer
import torch

from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
cuda = torch.device('cuda')


import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
!pip install yfinance
import yfinance as yf

from pandas.tseries.offsets import DateOffset
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# from transformers import BertForSequenceClassification, BertTokenizer
import torch

import time
import seaborn as sns

import nlpaug
import nlpaug.augmenter.word as naw

/opt/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jaimansukhani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


You should consider upgrading via the '/opt/miniconda3/bin/python -m pip install --upgrade pip' command.


In [20]:
!pip install contractions

You should consider upgrading via the '/opt/miniconda3/bin/python -m pip install --upgrade pip' command.


In [21]:
import contractions

In [22]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet, stopwords

nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jaimansukhani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jaimansukhani/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/jaimansukhani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/jaimansukhani/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jaimansukhani/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/jaimansukhani/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [23]:
# !pip3 install tensorflow

In [24]:
path = '../data/Transcripts'
df = pd.DataFrame()
companies = {'AMZN':'Amazon',
 'AAPL':'Apple',
 'INTC':'Intel',
 'MU':'Micron',
 'GOOGL': 'Alphabet',
 'MSFT': 'Microsoft',
 'AMD':'AMD',
 'ASML':'ASML',
 'NVDA':'NVIDIA',
 'CSCO':'Cisco'}

In [25]:
df = []
tf = 0
totalsummary = ''
totalqna = ''
for comp in os.listdir(path):
    if comp!='.DS_Store':
        company = comp
        files = os.listdir(path+'/'+company)
        for j in files:
            allinfo = []
            with open(path+'/'+company+'/'+j) as f:
                lines = f.readlines()
                summary = []
                qna = []
                i = 0
                while i<len(lines):
                    if lines[i]=='\n':
                        i+=1
                    elif lines[i][:3]=='===':
                        i+=1
                    elif 'operator' in lines[i].lower():
                        i+=3
                    elif lines[i][:3] == '---':
                        i+=1
                    elif 'presentation' in lines[i].lower():
                        i+=2
#                         print(j)
                        while i<len(lines):
#                             print(lines[i])
                            if '===' in lines[i]:
                                break
                            if 'Questions and Answers' in lines[i]:
                                break
                            elif lines[i]=='\n':
                                i+=1
                                continue
                            elif 'operator' in lines[i].lower():
                                i+=3
                            elif '---' in lines[i]:
                                i+=1
                            summary.append(lines[i].rstrip())
                            i+=1
                    
                    elif '[' in lines[i] and companies[company].lower() in lines[i].lower():
                        i+=2
                        while lines[i][:3]!='---':
                            if lines[i]=='\n':
                                i+=1
                                continue
                            summary.append(lines[i].rstrip())
                            i+=1
                    
                    elif 'Questions and Answers' in lines[i]:
                        i+=1
                        while i<len(lines):
                            if lines[i]=='\n':
                                i+=1
                            elif 'Operator' in lines[i]:
                                i+=2
                            elif lines[i][:3] == '---':
                                i+=1
                            elif '[' in lines[i]:
                                i+=2
                            else:
                                qna.append(lines[i].rstrip())
                                i+=1
                    
                    elif '[' in lines[i] and companies[company].lower() not in lines[i].lower():
                        while i<len(lines):
                            if lines[i]=='\n':
                                i+=1
                            elif 'Operator' in lines[i]:
                                i+=3
                            elif lines[i][:3] == '---':
                                i+=1
                            elif '[' in lines[i]:
                                i+=2
                            else:
                                qna.append(lines[i].rstrip())
                                i+=1
                    else:
                        i+=1
                
                if lines[0][0] == 'Q':
                    quarter = lines[0].split(' ')[0]
                    year = lines[0].split(' ')[1]
                else:
                    quarter = lines[1].split(' ')[0]
                    year = lines[1].split(' ')[1]
                cleansummary = []
#                 print(j)
                for i in range(len(summary)):
                    if '--' in summary[i]:
                        i+=1
                    elif 'Operator' in summary[i]:
                        i+=3
                    elif '[' in summary[i]:
#                         print(i, summary[i])
                        i+=1
                    else:
                        cleansummary.append(summary[i])
                summary_para = ' '.join(cleansummary).rstrip()
                qna_para = ' '.join(qna).rstrip()
                totalsummary += summary_para
                totalqna = qna_para
                allinfo.append(quarter)
                allinfo.append(year)
                allinfo.append(companies[company])
                allinfo.append(summary_para)
                allinfo.append(qna_para)
                df.append(allinfo)

In [26]:
earnings_transcript_df = pd.DataFrame(df)
earnings_transcript_df.columns =['Quarter','Year','Company','Summary','Q&A']

earnings_transcript_df

,Quarter,Year,Company,Summary,Q&A
0,Q4,2019,Amazon,"Hello, and welcome to our Q4 2019 f...",Great. Really appreciate this. Just ...
1,Q1,2016,Amazon,"Good day, everyone, and welcome to t...","Thanks a lot. Lots here, but interna..."
2,Q3,2019,Amazon,"Hello, and welcome to our Q3 2019 F...",I wanted to know if I can go a littl...
3,Q1,2020,Amazon,"For opening remarks, I will be turning the cal...","Great. First, I just wanted to ask, within the..."
4,Q1,2019,Amazon,"Hello, and welcome to our Q1 2019 f...",Okay. I'm going to ask 2 questions b...
...,...,...,...,...,...
183,Q1,2018,Cisco,Welcome to Cisco's First Quarter of ...,Our first question comes from the li...
184,Q3,2018,Cisco,Welcome to Cisco's Third Quarter Fis...,James Faucette from Morgan Stanley I...
185,Q3,2017,Cisco,Welcome to Cisco Systems' Third Quar...,I wanted to see if we could understa...
186,Q3,2020,Cisco,"Thanks, Michelle. Welcome, everyone, to Cisco...",Our first question comes from Paul Silverstein...


In [27]:
words = Counter(totalsummary.split(' '))
words += Counter(totalqna.split(' '))

In [28]:
def getAvgTxtLen(df):
    avg_length_summary_after_preprocessing = str(df['Summary'].str.len().mean()) 
    avg_length_qna_after_preprocessing = str(df['Q&A'].str.len().mean()) 
    print(avg_length_summary_after_preprocessing)
    print(avg_length_qna_after_preprocessing)

In [29]:
def strip_html_tags(text): 
    soup = BeautifulSoup(text, "html.parser") 
    stripped_text = soup.get_text(separator=" ") 
    return stripped_text 
    
# Make lowercase
earnings_transcript_df["Summary"] = earnings_transcript_df["Summary"].str.lower()
earnings_transcript_df["Q&A"] = earnings_transcript_df["Q&A"].str.lower()

# Remove URLS
earnings_transcript_df['Summary']= earnings_transcript_df['Summary'].str.replace('http(s)?:\/\/.*', '', regex=True)
earnings_transcript_df['Q&A']= earnings_transcript_df['Q&A'].str.replace('http(s)?:\/\/.*', '', regex=True)
print("Remove URLS")
getAvgTxtLen(earnings_transcript_df)

# remove HTML tags
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].apply(lambda x: strip_html_tags(x))
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].apply(lambda x: strip_html_tags(x))
print("remove HTML tags")
getAvgTxtLen(earnings_transcript_df)

# remove extra whitespaces
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].replace(r'/\s/\s+/g', ' ', regex=True)
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].replace(r'/\s/\s+/g', ' ', regex=True)
print("remove extra whitespaces")
getAvgTxtLen(earnings_transcript_df)

# remove contractions
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].apply(lambda x: ' '.join([contractions.fix(word) for word in x.split()]))
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].apply(lambda x: ' '.join([contractions.fix(word) for word in x.split()]))
print("remove contractions")
getAvgTxtLen(earnings_transcript_df)

# Remove characters following digits such as 1st, 2nd or 3rd
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].str.replace('\b\d+\w+\b', '', regex=True)
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].str.replace('\b\d+\w+\b', '', regex=True)
print("Remove characters following digits such as 1st, 2nd or 3rd")
getAvgTxtLen(earnings_transcript_df)

# Replace hyphens with space
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].replace('/\b-\b/g', ' ', regex=True)
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].replace('/\b-\b/g', ' ', regex=True)
print("Replace hyphens with space")
getAvgTxtLen(earnings_transcript_df)

# remove non alphanumeric characters
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].str.replace('[^a-zA-Z0-9 ]', ' ')
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].str.replace('[^a-zA-Z0-9 ]', ' ')
print("remove non alphabetical characters")
getAvgTxtLen(earnings_transcript_df)


Remove URLS
16292.622340425532
28507.35106382979
remove HTML tags
16292.617021276596
28088.063829787236
remove extra whitespaces
16292.617021276596
28088.063829787236
remove contractions
16065.97340425532
27840.127659574468
Remove characters following digits such as 1st, 2nd or 3rd
16065.97340425532
27840.127659574468
Replace hyphens with space
16065.97340425532
27840.127659574468
remove non alphabetical characters
16065.97340425532
27840.127659574468


In [30]:
stop_words = stopwords.words('english')
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words) ]))
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].astype('string')

earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words) ]))
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].astype('string')

print("Stopwords")
getAvgTxtLen(earnings_transcript_df)
# Lemmatize the Dataset
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_review(text):
    return ' '.join(lemmatizer.lemmatize(w,pos="v") for w in w_tokenizer.tokenize(text))

earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].apply(lemmatize_review)
earnings_transcript_df['Summary'] = earnings_transcript_df['Summary'].astype('string')
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].apply(lemmatize_review)
earnings_transcript_df['Q&A'] = earnings_transcript_df['Q&A'].astype('string')
print("Lemmatize")
getAvgTxtLen(earnings_transcript_df)



Stopwords
11871.898936170213
16657.393617021276
Lemmatize
11406.505319148937
15969.808510638299


In [31]:
avg_length_summary_after_preprocessing = str(earnings_transcript_df['Summary'].str.len().mean()) 
avg_length_qna_after_preprocessing = str(earnings_transcript_df['Q&A'].str.len().mean()) 
print(avg_length_summary_after_preprocessing)
print(avg_length_qna_after_preprocessing)

11406.505319148937
15969.808510638299


In [32]:
#df.iloc[0,3]
earnings_transcript_df

,Quarter,Year,Company,Summary,Q&A
0,Q4,2019,Amazon,hello welcome q4 2019 financial result confere...,great really appreciate aws business look sort...
1,Q1,2016,Amazon,good day everyone welcome amazon com q1 2016 f...,thank lot lot international retail revenue int...
2,Q3,2019,Amazon,hello welcome q3 2019 financial result confere...,want know go little bite deeper framework give...
3,Q1,2020,Amazon,open remark turn call director investor relati...,great first want ask within 4 billion covid re...
4,Q1,2019,Amazon,hello welcome q1 2019 financial result confere...,okay go ask 2 question first one really simple...
...,...,...,...,...,...
183,Q1,2018,Cisco,welcome cisco first quarter fiscal year 2018 f...,first question come line pierre ferragu sanfor...
184,Q3,2018,Cisco,welcome cisco third quarter fiscal year 2018 f...,jam faucette morgan stanley investment researc...
185,Q3,2017,Cisco,welcome cisco systems third quarter fiscal yea...,want see could understand guidance kelly chuck...
186,Q3,2020,Cisco,thank michelle welcome everyone cisco third qu...,first question come paul silverstein cowen com...


In [33]:
# Running augmentations
augdf = earnings_transcript_df.copy()
aug = naw.SynonymAug(aug_src='wordnet', model_path=None, name='Synonym_Aug', aug_min=10, aug_max=60, aug_p=0.3)
for i in range(len(augdf)):
    k = aug.augment(earnings_transcript_df.iloc[i]['Summary'],n=1)
    augdf['Summary'][i] = k[0]
    k = aug.augment(earnings_transcript_df.iloc[i]['Q&A'],n=1)
    augdf['Q&A'][i] = k[0]

earnings_transcript_df = earnings_transcript_df.append(augdf)
earnings_transcript_df = earnings_transcript_df.reset_index().drop(columns = ['index'])

In [34]:
today = pd.to_datetime("today")

# Consolidated Dataframe with sentiments for all companies
finance_df = pd.DataFrame(columns=["Company", "date", "Quarter", "Year", "percent change (between t-2 and t+2)","Sentiment"])

company_tickers = ['NVDA','MU','INTC','GOOGL','CSCO','ASML','AMZN','AMD','AAPL','MSFT']

price_change_perc_threshold_for_sentiment = 3.0

for ticker in company_tickers:
    
    # Get List of dates for Earnings call
    ticker_data = yf.Ticker(ticker)
    ticker_data.earnings_dates.reset_index(inplace=True)
    
    #Format Date column
    ticker_data.earnings_dates["Earnings Date"] = ticker_data.earnings_dates["Earnings Date"].apply(lambda x: x.strftime('%Y-%m-%d'))
    
    # Filter dates within window of our Transcripts
    earnings_dates_df = ticker_data.earnings_dates[(ticker_data.earnings_dates["Earnings Date"]<=today.strftime('%Y-%m-%d')) & (ticker_data.earnings_dates["Earnings Date"]>="2015-12-01")]
    earning_call_dates= earnings_dates_df.iloc[:,0]

    #print(earning_call_dates)
    
    prepared_df = pd.DataFrame(columns=["Company", "date", "Quarter", "Year", "percent change (between t-2 and t+2)","Sentiment"])
    
    historical_data = yf.download(ticker, start="2015-12-01", end="2022-11-21", interval="1d")

    historical_data.reset_index(inplace=True)

    for date in earning_call_dates:
        # Compute Quarters for each date to join finance df with Transcripts DF created in above cells
        month=pd.to_datetime(date).month
        year=pd.to_datetime(date).year
        if month<=3:
            quarter="Q1"
        elif month<=6:
            quarter="Q2"
        elif month<=9:
            quarter="Q3"
        else:
            quarter="Q4"
                
        # Compute % change by looking at T-2 and T+2 window from earnings call date
        # i.e. if Earnings call is 27 Oct 2022 => compute:
        # => (Closing_price['29 Oct'] - Closing_price['27 Oct']) * 100 / (Closing_price['27 Oct'])
        
        index= historical_data.index[historical_data["Date"]==pd.to_datetime(date)]
        tplus2= float(historical_data.loc[index+1,"Adj Close"])
        tminus2 = float(historical_data.loc[index-2,"Adj Close"])
        change = (tplus2 - tminus2)*100/tminus2
        
        # Computer sentiment based on price change percentage threshold
        if abs(change)< price_change_perc_threshold_for_sentiment:
            label="Neutral"
        elif change> price_change_perc_threshold_for_sentiment:
            label="Positive"
        else:
            label="Negative"
        prepared_df = prepared_df.append({"Company":companies[ticker],"date":date,"Quarter":quarter, "Year": year, "percent change (between t-2 and t+2)": change, "Sentiment": label}, ignore_index=True)
    prepared_df.reset_index(drop=True)
    prepared_df.to_csv("..\\data\\OHLC\\"+ticker+".csv", encoding='utf-8')
    finance_df=pd.concat([finance_df, prepared_df], ignore_index=True)
finance_df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Company,date,Quarter,Year,percent change (between t-2 and t+2),Sentiment
0,NVIDIA,2022-11-16,Q4,2022,-3.79257,Negative
1,NVIDIA,2022-08-24,Q3,2022,5.160274,Positive
2,NVIDIA,2022-05-25,Q2,2022,5.639725,Positive
3,NVIDIA,2022-02-16,Q1,2022,0.989006,Neutral
4,NVIDIA,2021-11-17,Q4,2021,5.495411,Positive
...,...,...,...,...,...,...
275,Microsoft,2017-01-26,Q1,2017,3.557926,Positive
276,Microsoft,2016-10-20,Q4,2016,3.468597,Positive
277,Microsoft,2016-07-19,Q3,2016,4.115477,Positive
278,Microsoft,2016-04-21,Q2,2016,-8.175201,Negative


In [35]:
finance_df.Year = finance_df.Year.astype('int')
earnings_transcript_df.Year = earnings_transcript_df.Year.astype('int')

In [36]:
df_merged = pd.merge(earnings_transcript_df, finance_df, on=['Company', 'Year', 'Quarter'], how='inner')
df_merged[df_merged['Company']=='Apple']

,Quarter,Year,Company,Summary,Q&A,date,percent change (between t-2 and t+2),Sentiment
36,Q2,2018,Apple,2q18 review c 1 result 1 new march qtr record ...,want ask thoughts sort iphone position couple ...,2018-05-01,8.778961,Positive
37,Q2,2018,Apple,2q18 inspection c one result 1 new march qtr r...,want ask thoughts sort iphone position couple ...,2018-05-01,8.778961,Positive
38,Q4,2019,Apple,4q19 review c 1 overview 1 co highest revenue ...,congratulations quarter iphone revenue traject...,2019-10-30,-0.116452,Neutral
39,Q4,2019,Apple,4q19 review c 1 overview 1 co highest revenue ...,congratulations quarter iphone revenue traject...,2019-10-30,-0.116452,Neutral
40,Q1,2016,Apple,1q16 business review c 1 highlight 1 time reco...,hello thank much term march quarter guidance i...,2016-01-26,-7.887974,Negative
41,Q1,2016,Apple,1q16 business review c 1 highlight 1 sentence ...,hello thank much term march quarter guidance i...,2016-01-26,-7.887974,Negative
42,Q3,2020,Apple,3q20 review c 1 highlight 1 uncertain environm...,tim light economic adversity talk prepare rema...,2020-07-30,13.948677,Positive
43,Q3,2020,Apple,3q20 review c 1 highlight 1 uncertain environm...,tim light economic adversity talk prepare rema...,2020-07-30,13.948677,Positive
44,Q3,2017,Apple,3q17 review c 1 highlight 1 unit revenue grow ...,luca first question gross margin guidance stro...,2017-08-01,5.11036,Positive
45,Q3,2017,Apple,3q17 review c 1 highlight 1 unit revenue grow ...,luca first question gross margin guidance stro...,2017-08-01,5.11036,Positive


In [44]:
df_merged.head()

,Quarter,Year,Company,Summary,Q&A,date,percent change (between t-2 and t+2),Sentiment
0,Q4,2019,Amazon,hello welcome q4 2019 financial result confere...,great really appreciate aws business look sort...,2019-10-24,-0.24919,Neutral
1,Q4,2019,Amazon,hello welcome q4 2019 financial result confere...,great really appreciate aws business look sort...,2019-10-24,-0.24919,Neutral
2,Q1,2016,Amazon,good day everyone welcome amazon com q1 2016 f...,thank lot lot international retail revenue int...,2016-01-28,-2.370061,Neutral
3,Q1,2016,Amazon,near day everyone welcome amazon com q1 2016 f...,thank lot lot international retail revenue int...,2016-01-28,-2.370061,Neutral
4,Q3,2019,Amazon,hello welcome q3 2019 financial result confere...,want know go little bite deeper framework give...,2019-07-25,-2.579112,Neutral


In [38]:
df_merged.to_csv("../data/OHLC/all_tickers.csv", encoding='utf-8')

In [43]:
df = df_merged
df = df[['Quarter','Year','Company','Summary','Q&A','Sentiment']]
df.head()

,Quarter,Year,Company,Summary,Q&A,Sentiment
0,Q4,2019,Amazon,hello welcome q4 2019 financial result confere...,great really appreciate aws business look sort...,Neutral
1,Q4,2019,Amazon,hello welcome q4 2019 financial result confere...,great really appreciate aws business look sort...,Neutral
2,Q1,2016,Amazon,good day everyone welcome amazon com q1 2016 f...,thank lot lot international retail revenue int...,Neutral
3,Q1,2016,Amazon,near day everyone welcome amazon com q1 2016 f...,thank lot lot international retail revenue int...,Neutral
4,Q3,2019,Amazon,hello welcome q3 2019 financial result confere...,want know go little bite deeper framework give...,Neutral


In [16]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_colwidth', None)

# For summary

## Sentiment Analyser

In [18]:
# df['no_contract'] = df['Summary'].apply(lambda x: [contractions.fix(word) for word in x.split()])
# df['no_contract'] = [' '.join(map(str, l)) for l in df['no_contract']]

In [19]:
# stopwords_list = stopwords.words('english')
# def ReviewProcessing(df):
#   #remove extra spaces
#     df['review_cleaned'] = df.no_contract.str.strip()
    
#   #remove html and url
#     df.review_cleaned = df.review_cleaned.str.replace(r'<[^<>]* />', '', regex=True)
#     df.review_cleaned = df.review_cleaned.apply(lambda x: re.sub(r"http\S+", "", x))
  
#   # lowercase
#     df.review_cleaned = df.review_cleaned.str.lower()
    
#   # remove non alphabetic 
#     df.review_cleaned = df.review_cleaned.apply(lambda x: re.sub("[^a-zA-Z]+", " ", x))
    
#   # split into list
#     df.review_cleaned = df.review_cleaned.str.split(' ')
    
#   # remove stop words 
#     df['review_stopwords'] = df.review_cleaned.apply(lambda x: [item for item in x if item not in stopwords_list])
#     return df 

In [20]:
# def get_wordnet_pos(word):
#     tag = nltk.pos_tag([word])[0][1][0].upper()
#     tag_dict = {"J": wordnet.ADJ,
#                 "N": wordnet.NOUN,
#                 "V": wordnet.VERB,
#                 "R": wordnet.ADV}

#     return tag_dict.get(tag, wordnet.NOUN)

# lemmatizer = nltk.stem.WordNetLemmatizer()
# def get_lemmatize(sent):
#     return " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sent)])

In [21]:
# import re
# clean_data = ReviewProcessing(df)

In [22]:
# clean_data.review_cleaned = clean_data.review_cleaned.apply(' '.join)
# clean_data.review_stopwords = clean_data.review_stopwords.apply(' '.join)
# clean_data['review_cleaned_lemmatized'] = clean_data.review_stopwords.apply(get_lemmatize)

In [151]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/jaimansukhani/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [152]:
# word_dict = {'manipulate':-1,'manipulative':-1,'jamescharlesiscancelled':-1,'jamescharlesisoverparty':-1,
#             'pedophile':-1,'pedo':-1,'cancel':-1,'cancelled':-1,'cancel culture':0.4,'teamtati':-1,'teamjames':1,
#             'teamjamescharles':1,'liar':-1}
word_dict = {'increase':1, 'increased':1, 'increases':1, "increasing":1, "increasingly":1, "expand":1,
'expanded':1, "expanding":1, "expands":1, "expansion":1, "expansions":1, "grow":1,
"grows":1, "grew":1, "growth":1, "growths":1, "improve":1, "improved":1, "improves":1,
"improvement":1, "improvements":1, "strong":1, "stronger":1, "strongest":1, "strongly":1, "decline":-40, 
"declined":-40, "declines":-40, "declining":-40, "deteriorate":-40, "deteriorates":-40,
"deteriorated":-40, "deteriorating":-40, "compress":-40, "compressed":-40, "compresses":-40,
"compressing":-40, "compressible":-40, "compression":-40, "reduce":-40, "reduces":-40,
"reduced":-40, "reducing":-40, "reduction":-40, "reductions":-40, "weak":-40, "weaker":-40,
"weakest":-40, "weaken":-40, "weakens":-40, "weakened":-40, "weakening":-40, "weakness":-40,
"weaknesses":-40}


sid.lexicon.update(word_dict)

list1 = []
for i in df['Summary']:
    list1.append((sid.polarity_scores(str(i)))['compound'])

In [153]:
df['score'] = pd.Series(list1)

In [154]:
def sentiment_category(sentiment):
    label = ''
    if(sentiment>0.9):
        label = 'Positive'
    elif(sentiment>0.6):
        label = 'Neutral'
    else:
        label = 'Negative'
    return(label)

df['sentiment_category_analyser'] = df['score'].apply(sentiment_category)

In [155]:
df.sentiment_category_analyser.value_counts()

Positive    203
Negative    157
Neutral       6
Name: sentiment_category_analyser, dtype: int64

In [156]:
c = 0
for i in range(len(df['Sentiment'])):
    if df['Sentiment'][i] == df['sentiment_category_analyser'][i]:
        c += 1
print("Accuracy: ", c/len(df['Sentiment']))
    

Accuracy:  0.3251366120218579


In [157]:
from sklearn.metrics import precision_score, recall_score, f1_score

print("Precision: ", precision_score(df['Sentiment'], df["sentiment_category_analyser"], average="macro"))
print("Recall: ", recall_score(df['Sentiment'], df["sentiment_category_analyser"], average="macro"))
print("F1 score: ", f1_score(df['Sentiment'], df["sentiment_category_analyser"], average="macro"))

Precision:  0.32487911337021813
Recall:  0.3586711711711712
F1 score:  0.27513060427673247


## Text blob

In [158]:
!pip install textblob
import textblob

You should consider upgrading via the '/opt/miniconda3/bin/python -m pip install --upgrade pip' command.


In [159]:
from textblob import TextBlob

# compute sentiment scores (polarity) and labels
sentiment_scores_tb = [round(TextBlob(article).sentiment.polarity, 2) for article in df['Summary']]
sentiment_category_tb = ['Positive' if score > 0 
                             else 'Negative' if score < 0 
                                 else 'Neutral' 
                                     for score in sentiment_scores_tb]

In [160]:
df['sentiment_category_tb'] = pd.DataFrame(sentiment_category_tb)

In [161]:
df.sentiment_category_tb.value_counts()

Positive    362
Negative      3
Neutral       1
Name: sentiment_category_tb, dtype: int64

In [162]:
c = 0
for i in range(len(df['Sentiment'])):
    if df['Sentiment'][i] == df['sentiment_category_tb'][i]:
        c += 1
print("Accuracy: ", c/len(df['Sentiment']))
    

Accuracy:  0.3524590163934426


In [163]:
from sklearn.metrics import precision_score, recall_score, f1_score

print("Precision: ", precision_score(df['Sentiment'], df["sentiment_category_tb"], average="macro"))
print("Recall: ", recall_score(df['Sentiment'], df["sentiment_category_tb"], average="macro"))
print("F1 score: ", f1_score(df['Sentiment'], df["sentiment_category_tb"], average="macro"))

Precision:  0.45119705340699817
Recall:  0.3355855855855856
F1 score:  0.17862393279459432


# For QnA 

## Sentiment Analyser

In [164]:
# word_dict = {'manipulate':-1,'manipulative':-1,'jamescharlesiscancelled':-1,'jamescharlesisoverparty':-1,
#             'pedophile':-1,'pedo':-1,'cancel':-1,'cancelled':-1,'cancel culture':0.4,'teamtati':-1,'teamjames':1,
#             'teamjamescharles':1,'liar':-1}
word_dict = {'increase':1, 'increased':1, 'increases':1, "increasing":1, "increasingly":1, "expand":1,
'expanded':1, "expanding":1, "expands":1, "expansion":1, "expansions":1, "grow":1,
"grows":1, "grew":1, "growth":1, "growths":1, "improve":1, "improved":1, "improves":1,
"improvement":1, "improvements":1, "strong":1, "stronger":1, "strongest":1, "strongly":1, "decline":-40, 
"declined":-40, "declines":-40, "declining":-40, "deteriorate":-40, "deteriorates":-40,
"deteriorated":-40, "deteriorating":-40, "compress":-40, "compressed":-40, "compresses":-40,
"compressing":-40, "compressible":-40, "compression":-40, "reduce":-40, "reduces":-40,
"reduced":-40, "reducing":-40, "reduction":-40, "reductions":-40, "weak":-40, "weaker":-40,
"weakest":-40, "weaken":-40, "weakens":-40, "weakened":-40, "weakening":-40, "weakness":-40,
"weaknesses":-40}


sid.lexicon.update(word_dict)

list1 = []
for i in df['Q&A']:
    list1.append((sid.polarity_scores(str(i)))['compound'])

In [165]:
df['score_qna'] = pd.Series(list1)

In [166]:
def sentiment_category(sentiment):
    label = ''
    if(sentiment>0.9):
        label = 'Positive'
    elif(sentiment>0.6):
        label = 'Neutral'
    else:
        label = 'Negative'
    return(label)

df['sentiment_category_analyser_qna'] = df['score_qna'].apply(sentiment_category)

In [167]:
df.sentiment_category_analyser_qna.value_counts()

Positive    302
Negative     60
Neutral       4
Name: sentiment_category_analyser_qna, dtype: int64

In [168]:
c = 0
for i in range(len(df['Sentiment'])):
    if df['Sentiment'][i] == df['sentiment_category_analyser_qna'][i]:
        c += 1
print("Accuracy: ", c/len(df['Sentiment']))
    

Accuracy:  0.32786885245901637


In [169]:
from sklearn.metrics import precision_score, recall_score, f1_score

print("Precision: ", precision_score(df['Sentiment'], df["sentiment_category_analyser_qna"], average="macro"))
print("Recall: ", recall_score(df['Sentiment'], df["sentiment_category_analyser_qna"], average="macro"))
print("F1 score: ", f1_score(df['Sentiment'], df["sentiment_category_analyser_qna"], average="macro"))

Precision:  0.21258278145695364
Recall:  0.33229166666666665
F1 score:  0.23813953488372094


## Text blob 

In [170]:
from textblob import TextBlob

# compute sentiment scores (polarity) and labels
sentiment_scores_tb = [round(TextBlob(article).sentiment.polarity, 2) for article in df['Q&A']]
sentiment_category_tb = ['Positive' if score > 0 
                             else 'Negative' if score < 0 
                                 else 'Neutral' 
                                     for score in sentiment_scores_tb]

In [171]:
df['sentiment_category_tb_qna'] = pd.DataFrame(sentiment_category_tb)

In [172]:
df.sentiment_category_tb_qna.value_counts()

Positive    366
Name: sentiment_category_tb_qna, dtype: int64

In [173]:
c = 0
for i in range(len(df['Sentiment'])):
    if df['Sentiment'][i] == df['sentiment_category_tb_qna'][i]:
        c += 1
print("Accuracy: ", c/len(df['Sentiment']))
    

Accuracy:  0.34972677595628415


In [174]:
from sklearn.metrics import precision_score, recall_score, f1_score

print("Precision: ", precision_score(df['Sentiment'], df["sentiment_category_tb_qna"], average="macro"))
print("Recall: ", recall_score(df['Sentiment'], df["sentiment_category_tb_qna"], average="macro"))
print("F1 score: ", f1_score(df['Sentiment'], df["sentiment_category_tb_qna"], average="macro"))

Precision:  0.11657559198542805
Recall:  0.3333333333333333
F1 score:  0.17273954116059378


/opt/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# #check presence of these words in text
# revenue = {'sales', 'revenue', 'top line', 'top bottom line', 'net revenue', 'organic revenue growth', 
#            'organic sales growth', 'operational sales'}
# earnings = {'eps', 'earnings', 'earnings per share',  'net income', 'bottom line', 'top bottom line'}
# profitibiality = {'margin', 'gross margin', 'operating margin', 'return invested capital', 'return capital'}

# Reference

https://www.kdnuggets.com/2018/08/emotion-sentiment-analysis-practitioners-guide-nlp-5.html

https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664